In [ ]:
# for google colaboratory
from os import path
import sys
if 'google' in sys.modules:
    files = ['LLD-icon-names.pkl', 'LLD-icon_PKL.zip', 'LLD-icon-sharp.hdf5', 
             'LLD-icon-sharp_indices.pkl']
    !pip install wget
    import wget
    import shutil
    from google.colab import drive
    drive.mount('/content/drive')
    !mkdir Data
    for f in files:
        if not path.isfile('Data/' + f):
            if path.isfile('/content/drive/My Drive/Colab/AFRO/' + f):
                shutil.copy('/content/drive/My Drive/Colab/AFRO/' + f, 'Data')
            else:
                wget.download('https://data.vision.ee.ethz.ch/sagea/lld/data/' + f, 'Data')
    !unzip -q -n Data/LLD-icon_PKL.zip -d Data

In [ ]:
# for debugging ------
from importlib import reload
# -------------------
import pickle
import h5py
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Open data

## icon-sharp

In [ ]:
files = h5py.File('Data/LLD-icon-sharp.hdf5', 'r')

In [ ]:
files.keys()

In [ ]:
clusters = files['labels/resnet/rc_128']
names = files['meta_data/names']
images = files['data']

### show clasters

In [ ]:
unique = len(set(clusters[()]))
samples = 8
fig, axs = plt.subplots(unique, samples, figsize=(10, 50))
for i in range(unique):
        indices = np.argwhere(clusters[()]==i)
        for j in range(8):
                axs[i,j].imshow(np.moveaxis(images[indices[j]], 1, -1).squeeze(0))
                axs[i,j].axis('off')

## icon_data

In [ ]:
with open('Data/LLD-icon/LLD-icon_data_0.pkl', 'rb') as f:
    data = pickle.load(f, encoding='latin1')

In [ ]:
data.shape

In [ ]:
plt.imshow(data[1])

In [ ]:
with open('Data/LLD-icon-names.pkl', 'rb') as f:
    labels = pickle.load(f)

In [ ]:
len(labels)

# Create text and add to icon

In [ ]:
import albumentations as A
import cv2 as cv
# custom module --- 
import utils
utils = reload(utils)
from utils import convert_to_pic
# ------------------

In [ ]:
img2 = convert_to_pic('McDonald')
tr = A.Resize(128, 128, cv.INTER_CUBIC)
img1 = tr(image=data[5])['image']

brows, bcols = img1.shape[:2]
rows,cols,channels = img2.shape

roi = img1[int(brows/2)-int(rows/2):int(brows/2)+int(rows/2), int(bcols/2)- 
int(cols/2):int(bcols/2)+int(cols/2) ]

img2gray = cv.cvtColor(img2,cv.COLOR_BGR2GRAY)
ret, mask = cv.threshold(img2gray, 10, 255, cv.THRESH_BINARY)
mask_inv = cv.bitwise_not(mask)

img1_bg = cv.bitwise_and(roi,roi,mask = mask_inv)

img2_fg = cv.bitwise_and(img2,img2,mask = mask)

dst = cv.add(img1_bg,img2_fg)
img1[int(brows/2)-int(rows/2):int(brows/2)+int(rows/2), int(bcols/2)- 
int(cols/2):int(bcols/2)+int(cols/2) ] = dst

In [ ]:
plt.imshow(dst)